In [1]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,PlayerElo
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_11425/3293468424.py:1: DtypeWarning: Columns (3,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'],


In [4]:
playerClasses, ratingsHistory = epochsElo(matches)

In [6]:
ratingsHistory_df = assembleDf(ratingsHistory)

In [7]:
# fill in missing values with last seen rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [14]:
def get_recent_rating_wp(ratingsHistory_df,tourney_date,winner_id,loser_id):
    if not type(tourney_date) == pd._libs.tslibs.timestamps.Timestamp:
        tourney_date = pd.to_datetime(tourney_date)
    timestamp = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
    winner_rating, loser_rating = ratingsHistory_df.loc[timestamp,[winner_id,loser_id]]
    wp = 1/(1 + 10**((loser_rating-winner_rating)/400))
    return winner_rating, loser_rating, wp

In [15]:
get_recent_rating_wp(ratingsHistory_df,pd.Timestamp('1877-07-09T00'),131867,131879)

(1500.0, 1500.0, 0.5)

In [9]:
padRow = pd.DataFrame({col: 1500.0 for col in ratingsHistory_df.columns}, index = [pd.Timestamp('1877-07-09T00')])
padRow

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0


In [10]:
ratingsHistory_df = pd.concat([padRow,ratingsHistory_df],axis=0)

In [20]:
we_le_pw = matches.apply(lambda x:
                           get_recent_rating_wp(
                               ratingsHistory_df, 
                               x['tourney_date'], 
                               x['winner_id'], 
                               x['loser_id']),
                            axis=1)

In [23]:
we_le_pw_df = pd.DataFrame.from_records(we_le_pw, columns = ['winner_elo','loser_elo','win_prob'])

In [24]:
we_le_pw_df

,winner_elo,loser_elo,win_prob
0,1500.000000,1500.000000,0.500000
1,1500.000000,1500.000000,0.500000
2,1500.000000,1500.000000,0.500000
3,1500.000000,1500.000000,0.500000
4,1500.000000,1500.000000,0.500000
...,...,...,...
876973,1520.131858,1489.986127,0.543275
876974,1510.276246,1353.839307,0.711057
876975,1416.488987,1456.834926,0.442197
876976,1522.338970,1495.882160,0.538001


In [25]:
singles_r_wp = pd.concat([singles_matches,we_le_pw_df],axis = 1)

In [26]:
singles_r_wp

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_elo,loser_elo,win_prob
0,1877-540,Wimbledon,Grass,32,G,1877-07-09,19,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000,0.500000
1,1877-540,Wimbledon,Grass,32,G,1877-07-09,21,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000,0.500000
2,1877-540,Wimbledon,Grass,32,G,1877-07-09,20,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000,0.500000
3,1877-540,Wimbledon,Grass,32,G,1877-07-09,18,113963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000,0.500000
4,1877-540,Wimbledon,Grass,32,G,1877-07-09,17,113999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876973,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,104,207987,6.0,NaN,...,NaN,NaN,NaN,716.0,31.0,966.0,12.0,1520.131858,1489.986127,0.543275
876974,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,103,144932,NaN,NaN,...,NaN,NaN,NaN,799.0,23.0,1117.0,8.0,1510.276246,1353.839307,0.711057
876975,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,102,209955,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1416.488987,1456.834926,0.442197
876976,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,205,209079,NaN,JE,...,NaN,NaN,NaN,1125.0,7.0,1548.0,2.0,1522.338970,1495.882160,0.538001


How well does the elo prediction match with reality?

In [29]:
singles_r_wp['win_prob'].mean() # the higher elo-rated player wins %56.4 of the time


0.5641973656273931

In [38]:
singles_r_wp[singles_r_wp['tourney_level']== 'G']['win_prob'].mean() 
# In grand-slam events, the higher rated player wins 58.1% of the time

0.5808836943239509

In [ ]:
singles_r_wp[singles_r_wp['tourney_level']== 'G']

In [33]:
singles_r_wp['tourney_level'].unique()

array(['G', 'D', 'A', 'M', 'C', 'F', 'S', '15', '25'], dtype=object)

In [51]:
np.mean(singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['winner_rank'] <singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['loser_rank'])

0.64775167086075

Of the ranked matches (both players have a ranking), 64.8% of the winners have a higher ranking.

In [52]:
np.mean(singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['loser_elo'] <singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['winner_elo'])

0.6332248240479825

In [53]:
# check that elos are not 1500...

In [ ]:
singles_matches[1:10].shape,matches[1:10].shape

In [ ]:
matches.dtypes

In [ ]:
testDf

In [ ]:
matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

In [ ]:
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

In [ ]:
testDf = singles_matches[1:10]
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)


In [ ]:
[1,2,3][0:-1]

In [ ]:
ratingsHistory_df

In [ ]:
singles_matches

In [ ]:
ratingsHistory_df.columns

In [ ]:
set(list(matches['winner_id'].unique()) + list(matches['loser_id'].unique()))

In [ ]:
set(ratingsHistory_df) - 

In [ ]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)

In [ ]:
ratingsHistory_df